# Redução de Dimensionalidade - Prof André Hochuli

Neste notebook exploraremos técnicas de redução de dimensionalidade, com foco em PCA (Principal Component Analysis) e t-SNE (t-Distributed Stochastic Neighbor Embedding).

Em cenários reais de Machine Learning, os dados frequentemente apresentam alta dimensionalidade, seja devido a grande número de atributos, vetores de características extraídos por CNNs ou embeddings latentes produzidos por autoencoders. Trabalhar diretamente nesses espaços pode:

* Aumentar custo computacional

* Introduzir redundância e correlação entre variáveis

* Dificultar análise exploratória e interpretação

* Sofrer com a maldição da dimensionalidade

**Objetivos desta notebook**

* Compreender o papel da redução dimensional no pipeline de ML

* Aplicar PCA como técnica linear baseada em decomposição espectral

* Aplicar t-SNE como técnica não linear voltada à preservação de estruturas locais

* Comparar visualmente os métodos em projeções bidimensionais

**Visão conceitual**

PCA: projeta os dados em direções ortogonais de máxima variância (autovetores da matriz de covariância), sendo útil para compressão, remoção de redundância e pré-processamento.

t-SNE: preserva vizinhanças locais ao minimizar a divergência KL entre distribuições de similaridade, sendo mais indicado para visualização e análise qualitativa de separabilidade de classes.

Ao longo da notebook, analisaremos os resultados tanto sob a perspectiva matemática quanto sob a ótica prática de análise exploratória de embeddings.

#Imports

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from sklearn.datasets import load_wine,load_breast_cancer,load_digits,make_classification, load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#Dataset Sintético

In [0]:
# ----------------------------
# Configuração do problema (sintético)
# ----------------------------
n_samples = 1000
n_features = 300
n_informative = 200      # Sinal real
n_redundant = 30        # Colinearidade
n_repeated = 0
n_classes = 3

synthetic = make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_informative=n_informative,
    n_redundant=n_redundant,
    n_repeated=n_repeated,
    n_classes=n_classes,
    n_clusters_per_class=2,
    class_sep=1.5,
    flip_y=0.01,
    random_state=42
)

#Carregando o dataset e normalizando

In [0]:
#X, y = load_wine(return_X_y=True)  
#X, y = load_iris(return_X_y=True)
#X, y = load_breast_cancer(return_X_y=True)  
X, y = load_digits(return_X_y=True)  
#X,y = synthetic



print(X.shape,y.shape)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)

components = np.arange(1, X_scaled.shape[1] + 1)

# Número de amostras
n_samples = X.shape[0]

# Número de features
n_features = X.shape[1]

# Classes únicas
classes = np.unique(y)

# Número de classes
n_classes = len(classes)

print(f"Número de amostras: {n_samples}")
print(f"Número de features: {n_features}")
print(f"Classes: {classes}")
print(f"Número de classes: {n_classes}")

#PCA 

### Principal Component Analysis (PCA)

O **PCA (Principal Component Analysis)** é uma técnica clássica de redução de dimensionalidade baseada em álgebra linear, cujo objetivo é projetar os dados em um novo sistema de coordenadas ortogonais que maximize a variância explicada.

Formalmente, o método parte da **matriz de covariância** dos dados padronizados e realiza sua decomposição espectral. Os **autovetores** definem as direções dos componentes principais, enquanto os **autovalores** quantificam a variância associada a cada componente. Os componentes são ordenados de forma decrescente pela variância explicada, permitindo selecionar apenas os mais informativos.

Do ponto de vista geométrico, o PCA realiza uma projeção linear no subespaço de maior energia, preservando ao máximo a dispersão global dos dados. Essa propriedade torna o método adequado para:

* Compressão de dados
* Remoção de redundância e correlação
* Redução de ruído
* Pré-processamento para classificadores

Além disso, a análise da **variância acumulada** fornece um critério objetivo para determinar o número mínimo de componentes necessários para representar adequadamente a estrutura do conjunto de dados.
__

##Inicialização

In [0]:
pca = PCA()
pca.fit(X_scaled)
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

## Variância Explicada

In [0]:
plt.figure(figsize=(10, 6))
# Barras - variância individual
plt.bar(components, explained_variance, alpha=0.7)

# Linha - variância acumulada
plt.plot(components, cumulative_variance, marker='o')

plt.xlabel("Número de Componentes Principais")
plt.ylabel("Variância Explicada")
plt.title("PCA - Variância Individual e Acumulada")
plt.xticks(components)
plt.ylim(0, 1.05)
plt.grid(True)
plt.tight_layout()
plt.show()

# ----------------------------
# Seleção (ex: 95%)
# ----------------------------
n_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Número mínimo de componentes para ≥95% da variância: {n_95}")


In [0]:
for i, var in enumerate(cumulative_variance):
    print(f"{i+1} componentes -> {var:.4f} variância acumulada")

# Número mínimo de componentes para 95% de variância
n_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nNúmero mínimo de componentes para ≥95% da variância: {n_95}")

## Visualização do Espaço (2D)

In [0]:
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

df_2d = pd.DataFrame({
    "PC1": X_pca_2d[:, 0],
    "PC2": X_pca_2d[:, 1],
    "Class": y
})

# ----------------------------
# Plot 2D com Seaborn
# ----------------------------
plt.figure()
sns.scatterplot(
    data=df_2d,
    x="PC1",
    y="PC2",
    hue="Class",
    palette="deep"
)

plt.title("PCA - Projeção 2D")
plt.tight_layout()
plt.show()

print("Variância explicada (2D):",
      np.sum(pca_2d.explained_variance_ratio_))

##Visualização do Espaço (3D)

In [0]:
# ----------------------------
# PCA 3D
# ----------------------------
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

df_3d = pd.DataFrame({
    "PC1": X_pca_3d[:, 0],
    "PC2": X_pca_3d[:, 1],
    "PC3": X_pca_3d[:, 2],
    "Class": y.astype(str)
})

# ----------------------------
# Paleta Seaborn
# ----------------------------
classes = df_3d["Class"].unique()
palette = sns.color_palette("deep", len(classes))
palette_hex = [mcolors.to_hex(c) for c in palette]
color_map = {cls: palette_hex[i] for i, cls in enumerate(classes)}

# ----------------------------
# Figura interativa
# ----------------------------
fig = go.Figure()

for cls in classes:
    subset = df_3d[df_3d["Class"] == cls]

    fig.add_trace(go.Scatter3d(
        x=subset["PC1"],
        y=subset["PC2"],
        z=subset["PC3"],
        mode='markers',
        marker=dict(
            size=4,
            color=color_map[cls],
            opacity=0.8
        ),
        name=f"Class {cls}"
    ))

fig.update_layout(
    title=f"PCA 3D Interativo (Var acumulada = {pca_3d.explained_variance_ratio_.sum():.3f})",
    template="plotly_white",
    paper_bgcolor="white",
    scene=dict(
        bgcolor="white",
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3"
    ),
    legend=dict(itemsizing='constant')
)

fig.show()
print("Variância explicada (3D):",
      np.sum(pca_3d.explained_variance_ratio_))

## Impacto da Variância Explicada (Componentes do PCA) na Classificação



A análise do impacto do número de componentes principais na performance dos classificadores permite avaliar o trade-off entre **compressão de informação** e **capacidade discriminativa**. No PCA, cada componente principal é ordenado pela variância explicada; entretanto, maior variância não implica necessariamente maior relevância para separação de classes.

Neste experimento, avaliou-se o desempenho de **SVC**, **KNN**, **Random Forest** e **Decision Tree** à medida que o número de componentes foi progressivamente alterado. O objetivo foi observar como a retenção de diferentes níveis de variância acumulada influencia a acurácia e a generalização dos modelos.

Os resultados evidenciam três comportamentos típicos:

1. **Região de sub-representação** (poucos componentes): perda de informação discriminativa, afetando principalmente modelos baseados em distância, como KNN e SVC.
2. **Região ótima intermediária**: número reduzido de componentes preserva estrutura relevante e reduz ruído, podendo inclusive melhorar generalização.
3. **Região de saturação**: aumento marginal de desempenho ao incluir componentes de baixa variância, que tendem a capturar ruído ou variações pouco informativas.

Observa-se ainda que:

* **SVC** tende a se beneficiar de uma representação compacta e bem estruturada.
* **KNN** é sensível à geometria do espaço projetado.
* **Random Forest** e **Decision Tree** são menos sensíveis à correlação original, mas podem se beneficiar da redução de redundância.

Assim, a variância acumulada deve ser interpretada não apenas como critério estatístico, mas como variável experimental no contexto supervisionado, sendo recomendável validar empiricamente o número ótimo de componentes para cada modelo.


In [0]:
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

# ----------------------------
# Função: treina e avalia para um alvo de variância
# ----------------------------
def train_eval(model_name, clf, var_target):
    if var_target == 1.0:
        pca = PCA(n_components=None)
        var_label = "100%"
    else:
        pca = PCA(n_components=var_target, svd_solver="full")
        var_label = f"{int(var_target*100)}%"

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("pca", pca),
        ("clf", clf)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    n_comp = pipe.named_steps["pca"].n_components_

    print(f"[{model_name}] var={int(var_target*100)}% | n_comp={n_comp:2d}/{X.shape[1]} | acc={acc:.4f}")

# ----------------------------
# Modelos (valores default razoáveis para demo)
# ----------------------------
models = {
    "SVM": SVC(kernel="rbf", C=10, gamma="scale", random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=7, weights="distance", p=2),
    "DecisionTree": DecisionTreeClassifier(random_state=42, max_depth=None),
    "RandomForest": RandomForestClassifier(
        n_estimators=300, random_state=42, n_jobs=-1, max_features="sqrt"
    )
}

# ----------------------------
# Rodar para 100%, 95%, 80%...
# ----------------------------
for name, clf in models.items():
    print(f"\n========== {name} ===========")
    for var_target in [1.00, 0.95, 0.80,.50,.30]:
        train_eval(name, clf, var_target)

#TSNE

##Estrutura Local e Estabilidade

Nesta etapa, utilizamos o **t-SNE** como ferramenta de visualização para investigar a preservação da **estrutura local** do espaço de características. Diferentemente do PCA, que realiza uma projeção linear orientada à variância global, o t-SNE modela probabilidades de vizinhança no espaço original e busca preservar essas relações no espaço reduzido por meio da minimização da divergência de Kullback–Leibler.

O foco desta análise não é desempenho classificatório, mas sim **qualidade estrutural da organização dos dados** em 2D, especialmente a formação de clusters e a separação qualitativa entre classes.

Para evidenciar uma propriedade importante do método, realizamos **seis execuções independentes**, variando a inicialização aleatória. Como o t-SNE envolve otimização não convexa, os resultados podem apresentar variações significativas entre execuções, mesmo mantendo os mesmos hiperparâmetros (perplexity, learning rate, número de iterações). Essa instabilidade se manifesta principalmente:

* Na orientação global dos clusters
* Na distância relativa entre grupos
* Na forma geométrica das regiões projetadas

Entretanto, observa-se que a **estrutura local intra-cluster tende a ser preservada**, reforçando o caráter exploratório da técnica.



In [0]:
y_str = y.astype(str)

# ----------------------------
# Configuração t-SNE 
# ----------------------------
perplexity = 30
n_iter = 1000
learning_rate = "auto"
init = "random" 

seeds = [0, 1, 2, 3, 4, 5]  # 6 execuções

# ----------------------------
# Plot grid (2x3)
# ----------------------------
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.ravel()

for ax, seed in zip(axes, seeds):
    tsne = TSNE(
        n_components=2,
        perplexity=perplexity,
        init=init,
        learning_rate=learning_rate,
        max_iter=n_iter,
        random_state=seed
    )
    Z = tsne.fit_transform(X_scaled)

    sns.scatterplot(
        x=Z[:, 0], y=Z[:, 1],
        hue=y_str,
        palette="deep",
        s=18, linewidth=0,
        ax=ax, legend=False
    )
    ax.set_title(f"t-SNE 2D | seed={seed}")
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle(f"Instabilidade do t-SNE: mesma entrada, seeds diferentes (init='{init}', perplexity={perplexity})", y=1.02)
plt.tight_layout()
plt.show()


## Comparação PCA vs t-SNE

Ao comparar com o PCA:

* **PCA** preserva estrutura global e variância máxima, produzindo projeções estáveis e determinísticas.
* **t-SNE** prioriza vizinhança local, frequentemente produzindo separações visuais mais evidentes entre grupos.
* PCA é adequado para pré-processamento e compressão.
* t-SNE é indicado para visualização e análise qualitativa de embeddings.

Essa comparação reforça que os métodos possuem objetivos distintos e devem ser utilizados de forma complementar no pipeline de análise.

In [0]:
import plotly.graph_objects as go

labels = y.astype(str)

# ----------------------------
# PCA 2D/3D
# ----------------------------
pca2 = PCA(n_components=2, random_state=42)
Z_pca2 = pca2.fit_transform(X_scaled)

pca3 = PCA(n_components=3, random_state=42)
Z_pca3 = pca3.fit_transform(X_scaled)

# ----------------------------
# t-SNE 2D/3D (visualização)
# ----------------------------
tsne2 = TSNE(
    n_components=2,
    perplexity=30,
    init="pca",
    learning_rate="auto",
    max_iter=1000,
    random_state=42
)
Z_tsne2 = tsne2.fit_transform(X_scaled)

tsne3 = TSNE(
    n_components=3,
    perplexity=30,
    init="pca",
    learning_rate="auto",
    max_iter=1000,
    random_state=42
)
Z_tsne3 = tsne3.fit_transform(X_scaled)

# ============================================================
# PAINEL 2D: PCA vs t-SNE (lado a lado)
# ============================================================
df2_pca = pd.DataFrame({"D1": Z_pca2[:, 0], "D2": Z_pca2[:, 1], "Class": labels})
df2_tsne = pd.DataFrame({"D1": Z_tsne2[:, 0], "D2": Z_tsne2[:, 1], "Class": labels})

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.scatterplot(
    data=df2_pca, x="D1", y="D2", hue="Class",
    palette="deep", s=18, linewidth=0, ax=axes[0], legend=True
)
axes[0].set_title(f"PCA 2D (var acumulada={pca2.explained_variance_ratio_.sum():.3f})")
axes[0].set_xticks([]); axes[0].set_yticks([])
axes[0].set_xlabel(""); axes[0].set_ylabel("")

sns.scatterplot(
    data=df2_tsne, x="D1", y="D2", hue="Class",
    palette="deep", s=18, linewidth=0, ax=axes[1], legend=False
)
axes[1].set_title("t-SNE 2D")
axes[1].set_xticks([]); axes[1].set_yticks([])
axes[1].set_xlabel(""); axes[1].set_ylabel("")

plt.suptitle("Comparativo 2D: PCA vs t-SNE", y=1.02)
plt.tight_layout()
plt.show()

# ============================================================
# PAINEL 3D: PCA vs t-SNE (lado a lado, interativo Plotly)
# ============================================================

# Paleta Seaborn -> HEX
classes = np.unique(labels)
palette = sns.color_palette("deep", len(classes))
palette_hex = [mcolors.to_hex(c) for c in palette]
color_map = {cls: palette_hex[i] for i, cls in enumerate(classes)}

# Cria subplots 3D lado a lado
from plotly.subplots import make_subplots
fig3 = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "scene"}, {"type": "scene"}]],
    subplot_titles=(
        f"PCA 3D (var acum={pca3.explained_variance_ratio_.sum():.3f})",
        "t-SNE 3D"
    )
)

# PCA 3D traces
for cls in classes:
    idx = (labels == cls)
    fig3.add_trace(
        go.Scatter3d(
            x=Z_pca3[idx, 0], y=Z_pca3[idx, 1], z=Z_pca3[idx, 2],
            mode="markers",
            marker=dict(size=3, color=color_map[cls]),
            name=f"Class {cls}",
            showlegend=True
        ),
        row=1, col=1
    )

# t-SNE 3D traces
for cls in classes:
    idx = (labels == cls)
    fig3.add_trace(
        go.Scatter3d(
            x=Z_tsne3[idx, 0], y=Z_tsne3[idx, 1], z=Z_tsne3[idx, 2],
            mode="markers",
            marker=dict(size=3, color=color_map[cls]),
            name=f"Class {cls}",
            showlegend=False  # legenda já aparece no painel da esquerda
        ),
        row=1, col=2
    )

# Fundo branco + ajustes
fig3.update_layout(
    title="Comparativo 3D Interativo: PCA vs t-SNE",
    template="plotly_white",
    paper_bgcolor="white",
    plot_bgcolor="white",
    height=600,
    legend=dict(itemsizing="constant")
)

fig3.update_scenes(
    bgcolor="white",
    xaxis=dict(showbackground=True, backgroundcolor="white"),
    yaxis=dict(showbackground=True, backgroundcolor="white"),
    zaxis=dict(showbackground=True, backgroundcolor="white"),
)

fig3.show()